In [ ]:
import nltk
nltk.download('punkt')
import json
import pickle
import numpy as np
import nltk
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.optimizers import Adam
from nltk.stem import SnowballStemmer


stemmer = SnowballStemmer ('spanish')
ignorar_palabras = ["?", "¿", "!", "¡"]
archivos_datos = open("intenciones.json").read()
intenciones = json.load(archivos_datos)

def tokenizador():
    palabras = []
    clases = []
    documentos = []

    for intencion in intenciones ["intenciones"]:
      for patron in intencion ["patrones"]:
        w = nltk.word_tokenize(patron)
        palabras.extend(w)
        documentos.append((w, intencion["categoria"]))
        if intencion["categoria"] not in clases:
            clases.append(intencion["categoria"])

    return palabras, clases, documentos

def lamantizador(palabras, clases, documentos):
    palabras = [stemmer.stem(w.lower()) for w in palabras if w not in ignorar_palabras]
    palabras2 = palabras
    pickle.dump(palabras, open("palabras.pkl", "wb"))
    pickle.dump(clases, open("classes.pkl", "wb"))
  
    return palabras2

def entrenamiento (palabras, clases, documentos):
    entrenamiento = []
    salida_vacia = [0] * len(clases)
    for doc in documentos:
      bolsa = []
      palabras_patron = doc[0]
      palabras_patron = [stemmer.stem(word.lower()) for word in palabras_patron if word not in ignorar_palabras]
      for w in palabras:
        bolsa.append(1) if w in palabras_patron else bolsa.append(0)
        salida_fila = list (salida_fila)
        salida_fila [clases.index(doc[1])] = 1
        entrenamiento = np.array(entrenamiento, dtype = object)
        x_entrenamiento = list(entrenamiento[:,0])
        y_entrenamiento = list(entrenamiento[:,1])
        return x_entrenamiento, y_entrenamiento

def contruir_modelo(x_entrenamiento, y_entrenamiento, clases):
    modelo = Sequential()
    modelo.add(Dense(128, input_shape = (len(x_entrenamiento[0]),), activation='relu'))
    modelo.add(Dropout(0.5))
    modelo.add(Dense(64, activation='relu'))
    modelo.add(Dropout(0.5))
    modelo.add(Dense(len(clases), activation = 'softmax'))
    modelo.compile(loss='categorical_crossentropy', optimizer=Adam(), metrics=['accuracy'])
    historia = modelo.fit(np.array(x_entrenamiento), np.array(y_entrenamiento), epochs=50, batch_size = 5, verbose=1, validation_split=0.16)

    import matplotlib.pyplot as plt

    #Grafica de los valores de precisión de entrenamiento y validación
    plt.plot(historia.history['accuracy'])
    plt.plot(historia.history['val_accuracy'])
    plt.title('Modelo accuracy')
    plt.ylabel('Accuracy')
    plt.xlabel('Epoch')
    plt.legend(['Entrenamiento', 'Prueba'], loc = 'upper left')
    plt.show()
    
    #Grafica de valores de las pérdidas de entrenamiento y validación
    plt.plot(historia.history['loss'])
    plt.plot(historia.history['val_loss'])
    plt.title('Modelo loss')
    plt.ylabel('Loss')
    plt.xlabel('Epoch')
    plt.legend(['Entrenamiento', 'Prueba'], loc='upper left')

    #construye el modelo y lo guarda
    modelo.save('modelo.h5')
    return modelo

    x_entrenamiento, y_entrenamiento = entrenamiento(lamentizador(tokenizador()))
    constrir_modelo(x_entrenamiento, y_entrenamiento, lamentizador(tokenizador())[1])
